In [1]:
import pandas as pd
import numpy as np
from scipy import stats

# Define the file names
file_bcf = 'BCF.xlsx'
file_ps_bart = 'ps_BART.xlsx'

# Function to calculate Cohen's d
def calculate_cohens_d(group1, group2):
    """
    Calculates Cohen's d for two independent samples.
    """
    n1, n2 = len(group1), len(group2)
    var1, var2 = np.var(group1, ddof=1), np.var(group2, ddof=1)
    
    # Calculate Pooled Standard Deviation
    # Formula: sqrt( ((n1-1)*s1^2 + (n2-1)*s2^2) / (n1+n2-2) )
    numerator = ((n1 - 1) * var1) + ((n2 - 1) * var2)
    denominator = n1 + n2 - 2
    
    if denominator == 0: return 0 # Avoid division by zero for single samples
    
    pooled_sd = np.sqrt(numerator / denominator)
    
    if pooled_sd == 0: return 0 # Avoid division by zero if variance is 0
    
    # Calculate Cohen's d
    d = (np.mean(group1) - np.mean(group2)) / pooled_sd
    return d

def interpret_effect_size(d):
    """
    Returns a string interpretation of Cohen's d magnitude.
    """
    abs_d = abs(d)
    if abs_d < 0.2: return "Negligible"
    elif abs_d < 0.5: return "Small"
    elif abs_d < 0.8: return "Medium"
    else: return "Large"

try:
    # 1. Load the dataframes
    df_bcf = pd.read_excel(file_bcf)
    df_ps_bart = pd.read_excel(file_ps_bart)
    
    print(f"Loaded {file_bcf} (Rows: {len(df_bcf)})")
    print(f"Loaded {file_ps_bart} (Rows: {len(df_ps_bart)})\n")

    # 2. Identify common columns
    common_columns = list(set(df_bcf.columns) & set(df_ps_bart.columns))
    
    if not common_columns:
        print("Error: No common columns found between the two files.")
    else:
        results = []

        # 3. Analyze each column
        for col in common_columns:
            # Extract data and drop missing values
            sample1 = df_bcf[col].dropna() # BCF Group
            sample2 = df_ps_bart[col].dropna() # ps-BART Group
            
            # Ensure sufficient data
            if len(sample1) < 2 or len(sample2) < 2:
                continue

            # A. Perform Welch's t-test (equal_var=False)
            t_statistic, p_value = stats.ttest_ind(sample1, sample2, equal_var=False)
            
            # B. Calculate Cohen's d
            d_value = calculate_cohens_d(sample1, sample2)
            magnitude = interpret_effect_size(d_value)
            
            # C. Store results
            results.append({
                'Column': col,
                'BCF_Mean': round(sample1.mean(), 4),
                'ps_BART_Mean': round(sample2.mean(), 4),
                'P_Value': round(p_value, 4),
                'Significant (0.05)': 'Yes' if p_value < 0.05 else 'No',
                "Cohen's d": round(d_value, 4),
                'Effect Size': magnitude
            })

        # 4. Create DataFrame and Display
        results_df = pd.DataFrame(results)
        
        # Sort by Column name for easier reading
        results_df = results_df.sort_values(by='Column')

        print("-" * 100)
        print("Statistical Comparison: BCF vs ps-BART")
        print("-" * 100)
        
        # Adjust display options to make sure columns aren't hidden
        pd.set_option('display.max_columns', None)
        pd.set_option('display.width', 1000)
        
        print(results_df.to_string(index=False))
        print("-" * 100)
        
        # Save results to a new Excel file for easy viewing
        output_filename = 'Comparison_Results_With_Effect_Size.xlsx'
        results_df.to_excel(output_filename, index=False)
        print(f"\nDetailed results saved to '{output_filename}'")

except FileNotFoundError as e:
    print(f"Error: File not found - {e.filename}")
except Exception as e:
    print(f"An unexpected error occurred: {e}")

Loaded BCF.xlsx (Rows: 100)
Loaded ps_BART.xlsx (Rows: 100)

----------------------------------------------------------------------------------------------------
Statistical Comparison: BCF vs ps-BART
----------------------------------------------------------------------------------------------------
                Column  BCF_Mean  ps_BART_Mean  P_Value Significant (0.05)  Cohen's d Effect Size
     empirical_ATE_MAE    0.2237        0.1985   0.0277                Yes     0.3141       Small
    empirical_ATE_MAPE    0.1299        0.1239   0.3784                 No     0.1249  Negligible
    empirical_ATE_RMSE    0.3414        0.2335   0.0000                Yes     1.2446       Large
   empirical_ATE_cover    0.8651        0.9918   0.0000                Yes    -1.2057       Large
     empirical_ATE_len    0.7751        1.2177   0.0000                Yes    -4.9513       Large
  theoretical_CATE_MAE    1.5141        0.6046   0.0000                Yes     1.6614       Large
 theoretical